# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [41]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import sys
%matplotlib inline

# Import API key
from api_keys import g_key
#from api_keys import weather_api_key


# Data File path (CSV)
data = "./output_data/cities.csv"

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [42]:
#read in csv file
Vacation_City_df = pd.read_csv(data)

Vacation_City_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
0,0,Srednekolymsk,67.45,153.68,-10.08,90,97,8.05,RU,1605588318
1,1,Araceli,10.55,119.99,83.89,80,77,15.14,PH,1605588318
2,2,Touros,-5.20,-35.46,77.00,88,20,4.70,BR,1605588318
3,3,Erzin,36.96,36.20,50.00,100,0,4.21,TR,1605588319
4,4,Cherskiy,68.75,161.30,-11.49,90,100,6.67,RU,1605588319


In [43]:
#set variables for heatmap
locations = Vacation_City_df[["Latitude", "Longitude"]].astype(float)
Humidity = Vacation_City_df["Humidity"].astype(float)

In [44]:
#configure heatmap
gmaps.configure(api_key=g_key)

fig= gmaps.figure(center=[0,0], zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, max_intensity=100,
                                point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
#create new df with ideal vacation cities using ideal criteria
Ideal_criteria_df = Vacation_City_df.loc[(Vacation_City_df["Temperature"]<95) & (Vacation_City_df["Temperature"]>75)
                                          & (Vacation_City_df["Wind Speed"]<15) & (Vacation_City_df["Wind Speed"]>5)
                                          & (Vacation_City_df["Cloudiness"]==0)].dropna()
Ideal_criteria_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
112,112,Yulara,-25.24,130.99,91.40,10,0,8.05,AU,1605588343
183,183,Cuamba,-14.80,36.54,76.82,60,0,8.23,MZ,1605588358
257,257,Arona,28.10,-16.68,75.20,31,0,9.17,ES,1605588370
277,277,Lar,26.20,83.97,82.67,37,0,10.54,IN,1605588374
291,291,Broken Hill,-31.95,141.43,77.00,17,0,8.05,AU,1605588376
292,292,Navrongo,10.90,-1.09,75.06,30,0,7.16,GH,1605588207
480,480,Sheopur,25.67,76.70,79.20,47,0,5.70,IN,1605588411
491,491,Pyapon,16.28,95.68,86.09,66,0,6.73,MM,1605588413
546,546,Fomboni,-12.28,43.74,79.95,80,0,11.12,KM,1605588423


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
#created new df with only ideal vacation cities and add Hotel name to df
hotel_df= Ideal_criteria_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel"]= ""
#remove additional index
hotel_df.reindex()
hotel_df

,City,Country,Latitude,Longitude,Hotel
112,Yulara,AU,-25.24,130.99,
183,Cuamba,MZ,-14.80,36.54,
257,Arona,ES,28.10,-16.68,
277,Lar,IN,26.20,83.97,
291,Broken Hill,AU,-31.95,141.43,
292,Navrongo,GH,10.90,-1.09,
480,Sheopur,IN,25.67,76.70,
491,Pyapon,MM,16.28,95.68,
546,Fomboni,KM,-12.28,43.74,


In [47]:
#set variables to use in params
target_type = "lodging"
radius = 5000

#write params dict
params = {
    "types": target_type,
    "radius": radius,
    "key": g_key
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#iterate through df to get lat and lng for hotels
for index, row in hotel_df.iterrows():

    # get hotel info from df
    lat = row['Latitude']
    long = row['Longitude']

    # add keyword to params dict
    params['location'] = f"{lat},{long}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {'City'}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

hotel_df


Retrieving Results for Index 112: City.
Closest hotel is Desert Gardens Hotel - Ayers Rock Resort.
------------
Retrieving Results for Index 183: City.
Closest hotel is Hotel Vision.
------------
Retrieving Results for Index 257: City.
Closest hotel is Paradise Court.
------------
Retrieving Results for Index 277: City.
Closest hotel is Ovaiskarni ward Lar.
------------
Retrieving Results for Index 291: City.
Closest hotel is Daydream Motel & Cottages.
------------
Retrieving Results for Index 292: City.
Closest hotel is Azakka Hotel.
------------
Retrieving Results for Index 480: City.
Closest hotel is Hotel Bismillah.
------------
Retrieving Results for Index 491: City.
Closest hotel is Aung Naing Thu Motel.
------------
Retrieving Results for Index 546: City.
Closest hotel is Auberge les abou.
------------


,City,Country,Latitude,Longitude,Hotel,Hotel Name
112,Yulara,AU,-25.24,130.99,,Desert Gardens Hotel - Ayers Rock Resort
183,Cuamba,MZ,-14.80,36.54,,Hotel Vision
257,Arona,ES,28.10,-16.68,,Paradise Court
277,Lar,IN,26.20,83.97,,Ovaiskarni ward Lar
291,Broken Hill,AU,-31.95,141.43,,Daydream Motel & Cottages
292,Navrongo,GH,10.90,-1.09,,Azakka Hotel
480,Sheopur,IN,25.67,76.70,,Hotel Bismillah
491,Pyapon,MM,16.28,95.68,,Aung Naing Thu Motel
546,Fomboni,KM,-12.28,43.74,,Auberge les abou


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Latitude", "Longitude"]]


In [49]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)

fig= gmaps.figure(center=[0,0], zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, max_intensity=100,
                                point_radius=5)

fig.add_layer(heat_layer)


# Create a marker_layer using the hotel name list to fill the info box
markers = gmaps.marker_layer(marker_locations,hotel_info)
fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))